In [1]:
user = "HOME"
pathtorepo = "C:\\Users\\" * user * "\\Desktop\\"

using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")

  Activating project at `C:\Users\HOME\Desktop\dynamical-systems\env\integrate`


In [2]:
using StaticArrays, DifferentialEquations, DynamicalSystems
using CairoMakie

In [3]:
function HR_mem(u, p, t)
    function sigma(x)
        return 1.0 / ( 1.0 + exp( -10.0 * ( x  - ( - 0.25 ) ) ) )
    end
    memristor(z, k1_me, k2_me) = k1_me + k1_me * z^2

    a, b, c, d, s, xr, r,  I, vs, k1, k2, k1_me, k2_me  = p
    x1, y1, z1, x2, y2, z2, z = u
    
    du1 = y1 + b * x1 ^ 2 - a * x1 ^3 - z1 + I - k1 * ( x1 - vs ) * sigma(x2) + memristor(z, k1_me, k2_me)*(x2 - x1)
    du2 = c - d * x1 ^2 - y1
    du3 = r * ( s * ( x1 - xr ) - z1 )
    
    du4 = y2 + b * x2 ^ 2 - a * x2 ^3 - z2 + I - k2 * ( x2 - vs ) * sigma(x1) + memristor(z, k1_me, k2_me)*(x1 - x2)
    du5 = c - d * x2 ^2 - y2
    du6 = r * ( s * ( x2 - xr ) - z2 )

    du7 = x1 - x2
    
    return SVector(du1, du2, du3, du4, du5, du6, du7)
end

HR_mem (generic function with 1 method)

In [4]:
function output(idx_k1_me, idx_k2_me, k1_me, k2_me, u0)
    println("Индекс k1_me: $idx_k1_me; k1_me: $k1_me");flush(stdout)
    println("Индекс k2_me: $idx_k2_me; k2_me: $k2_me");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output(idx_k1_me, k1_me, u0)
    println("Индекс k1_me: $idx_k1_me; k1_me: $k1_me");flush(stdout)
    println("Начальные условия: $u0");flush(stdout)
end
function output_end_iter(ΛΛ, u0_lc)
    println("Спектр: $(ΛΛ)");flush(stdout)
    println("Последняя точка: $(u0_lc)");flush(stdout)
end
function separate()
    println();flush(stdout)
    println("---------");flush(stdout)
end;

In [5]:
t = 500; tt = 250; tstep = 0.001
time_attract = 250.0

trange = range(0.0, t, step = tstep)
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [6]:
a = 1.; b = 3.; c = 1.; d = 5.;
xr = -1.6; r = 0.01; s = 5.; I = 4.; xv = 2.;
k1= -0.4; k2 = -0.4;
k1_me = 0.00
k2_me = 0.0
p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me, k2_me];

In [7]:
u0 = [-1.5, 0.0, 0.0, -2.5, 0.0, 0.0, 0.0]

7-element Vector{Float64}:
 -1.5
  0.0
  0.0
 -2.5
  0.0
  0.0
  0.0

In [8]:
k1_merange = range( 0.0,  1.0, length = 51 )
k2_merange = range( 0.0,  1.0, length = 51)

0.0:0.02:1.0

In [9]:
Λs = zeros(length(k1_merange), length(k2_merange), 7)
u0s = zeros(length(k1_merange), length(k2_merange), 7);

In [10]:
const k1_me_start_value = k1_merange[1]

function init_ds(k2_me, u0_lc)

    #p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_start_value]
    #ds = ContinuousDynamicalSystem(TM, u0_lc, p)

    p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me_start_value, k2_me];
    ds = CoupledODEs(HR_mem, u0_lc, p, diffeq = integ_set)
    return ds
end

function init_ds(k1_me, k2_me, u0_lc)
    #p = SA[α, τ, τD, τy, J, xthr, ythr, U0_, ΔU0, β, I0_]
    #ds = ContinuousDynamicalSystem(TM, u0_lc, p)

    p = [a, b, c, d, s, xr, r, I, xv, k1, k2, k1_me, k2_me];
    ds = CoupledODEs(HR_mem, u0_lc, p, diffeq = integ_set)
    return ds
end
function goto_attractor(ds_, time_attract)
    #tr = trajectory(ds_, time_attract; Ttr = tt, diffeq = integ_set)

    tr, _ = trajectory(ds_, time_attract; Δt = integ_set.dt, Ttr = tt)

    u0_lc = tr[end]
    return u0_lc
end
function spectrum(ds_)
    ΛΛ = lyapunovspectrum(ds_, t)
    return ΛΛ
end
function save_output(idx_k1_me, ΛΛ, u0_lc)
    Λs[1, idx_k1_me, :] = ΛΛ
    u0s[1, idx_k1_me, :] = u0_lc
end
function save_output(index_k1_me, idx_k2_me, ΛΛ, u0_lc)
    Λs[index_k1_me, idx_k2_me, :] = ΛΛ
    u0s[index_k1_me, idx_k2_me, :] = u0_lc
end
function save_tofile()
    save("LSs_50x50.jld", "data", Λs)
    save("u0s_50x50.jld", "data", u0s)
end

save_tofile (generic function with 1 method)

In [11]:
for (idx_k2_me, k2_me_) in enumerate(k1_merange)
    
    if idx_k2_me == 1 # Проверка на первую итерацию
        # Если использовать while условие не нужно!
        global u0_lc = u0
    end
    
    #output(idx_k1_me, k1_me_, u0_lc)
    
    ds = init_ds(k2_me_, u0_lc) # Инициализация системы
    u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
    ds = init_ds(k2_me_, u0_lc)
    ΛΛ = spectrum(ds) # Вычисление спектра
    
    #output_end_iter(ΛΛ, u0_lc)
    
    save_output(idx_k2_me, ΛΛ, u0_lc) # Сохранение значений
    save_tofile()
    
    # separate()
end

In [12]:
for (idx_k2_me, k2_me_) in enumerate(k1_merange)
    for (idx_k1_me, k1_me_) in enumerate(k2_merange)
        
        if idx_k1_me == 1 # Если использовать while условие не нужно!
            continue
        end
        
        
        u0_lc = u0s[idx_k1_me - 1, idx_k2_me, :]
        
        # output(idx_I0, idx_U0, I0_, U0_, u0_lc)
        
        ds = init_ds(k1_me_, k2_me_, u0_lc)
        u0_lc = goto_attractor(ds, time_attract) # Выход на аттрактор
        ds = init_ds(k1_me_, k2_me_, u0_lc)
        ΛΛ = spectrum(ds) # Вычисление спектра
        
        # output_end_iter(ΛΛ, u0_lc)
        
        save_output(idx_k1_me, idx_k2_me, ΛΛ, u0_lc)
        save_tofile()
        
        # separate()
    end
end